分词###

In [1]:
from transformers import BertTokenizer, BertModel, BertForMaskedLM
import smoothnlp as snlp
import re
import torch

In [2]:
sentences = []

ubuntu_path = "/home/zoe/文档/6.10/train.txt"
win_path = "D:/数据集大荟萃/DIY/train.txt"

with open(ubuntu_path,'r',encoding="utf8") as f:
  sentences = f.readlines()

max_sent_len = 512

import smoothnlp as snlp
import re
words_list = []
words_str = []
sents_list = []
masked_idxs_list = []
for sent in sentences:
    sent = sent.replace('\ufeff','')
    sent = sent.replace(' ','')
    sent = sent[1:]
    sent = sent.strip()
    sents_list.append(sent)

    words = snlp.segment(sent)
    masked_idxs = []
    for i,word in enumerate(words):
        if word=='#':
            words[i]='[MASK]'
            masked_idxs.append(i+1)
    masked_idxs_list.append(masked_idxs)
    print(sent)
    print(len(sent))
    
    words = ['[CLS]'] + words + ['[SEP]']
    print(len(words))
    if len(sent)<max_sent_len:
        print((max_sent_len-len(sent)-2))
        words = words + (max_sent_len-len(sent)-2)*["[PAD]"]
        print(len(words))
    words_list.append(words)
    words_str.append(" ".join(words))

###（1898年3月5日-1976年1月8日），字翔宇，曾用名飞飞、伍豪、少山、冠生等，原籍浙江绍兴，1898年3月5日生于江苏淮安。1921年加入中国共产党，是伟大的马克思主义者，伟大的无产阶级革命家、政治家、军事家、外交家，党和国家主要领导人之一，中国人民解放军主要创建人之一，中华人民共和国的开国元勋，是以毛泽东同志为核心的党的第一代中央领导集体的重要成员。
183
103
327
430
周恩来（1898年3月5日-1976年1月8日），字翔宇，曾用名飞飞、伍豪、少山、冠生等，原籍浙江绍兴，1898年3月5日生于江苏淮安。1921年加入中国共产党，是伟大的马克思主义者，伟大的无产阶级革命家、政治家、军事家、外交家，党和国家主要领导人之一，中国人民解放军主要创建人之一，中华人民共和国的开国元勋，是以###同志为核心的党的第一代中央领导集体的重要成员。
183
104
327
431
周恩来（1898年3月5日-1976年1月8日），字翔宇，曾用名飞飞、伍豪、少山、冠生等，原籍####，1898年3月5日生于江苏淮安。1921年加入中国共产党，是伟大的马克思主义者，伟大的无产阶级革命家、政治家、军事家、外交家，党和国家主要领导人之一，中国人民解放军主要创建人之一，中华人民共和国的开国元勋，是以毛泽东同志为核心的党的第一代中央领导集体的重要成员。
183
104
327
431
周恩来（1898年3月5日-1976年1月8日），字翔宇，曾用名飞飞、伍豪、少山、冠生等，原籍浙江绍兴，1898年3月5日生于####。1921年加入中国共产党，是伟大的马克思主义者，伟大的无产阶级革命家、政治家、军事家、外交家，党和国家主要领导人之一，中国人民解放军主要创建人之一，中华人民共和国的开国元勋，是以毛泽东同志为核心的党的第一代中央领导集体的重要成员。
183
104
327
431
周恩来（1898年3月5日-1976年1月8日），字翔宇，曾用名飞飞、伍豪、少山、冠生等，原籍浙江绍兴，1898年3月5日生于江苏淮安。1921年加入中国###，是伟大的马克思主义者，伟大的无产阶级革命家、政治家、军事家、外交家，党和国家主要领导人之一，中国人民解放军主要创建人之一，中华人民共和国的开国元勋，是以毛泽东同志为核心的党的第一代中央领导集体的重要成员。
183
104
327
431


In [3]:
#加载词典
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

all_tokens_tensor = []
all_segments_tensors = []
for word_str in words_list:
    # 将 token 转为 vocabulary 索引
    indexed_tokens = tokenizer.convert_tokens_to_ids(word_str)

    # 定义句子 A、B 索引
    segments_ids = [0]*len(word_str)
    
    # 将 inputs 转为 PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    
    all_tokens_tensor.append(tokens_tensor)
    all_segments_tensors.append(segments_tensors)

In [4]:
# 加载模型 pre-trained model (weights)
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval()
 
for tokens_tensor,segments_tensors,masked_idxs in zip(all_tokens_tensor,all_segments_tensors,masked_idxs_list):
    # cuda
    tokens_tensor = tokens_tensor.to('cuda')
    segments_tensors = segments_tensors.to('cuda')
    model.to('cuda')

    # Predict all tokens
    with torch.no_grad():
        predictions = model(tokens_tensor, segments_tensors)

    # confirm we were able to predict 'henson'
    #predicted_index = torch.argmax(predictions[0, masked_index]).item()
    predicted_token = tokenizer.convert_ids_to_tokens(masked_idxs)
    print(predicted_token)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


['[unused1]', '[unused2]', '[unused3]']
['[unused85]', '[unused86]', '[unused87]']
['[unused27]', '[unused28]', '[unused29]', '[unused30]']
['[unused34]', '[unused35]', '[unused36]', '[unused37]']
['[unused40]', '[unused41]', '[unused42]']
['[unused1]', '[unused2]', '[unused3]']
['[unused15]', '[unused16]', '[unused17]', '[unused18]']
['[unused29]', '[unused30]', '[unused31]']
['[unused18]', '[unused19]', '[unused20]', '[unused21]']


In [6]:
print(words_str[0])

[CLS] [MASK] [MASK] [MASK] （ 1898年 3月5日 - 1976年 1月8日 ） ， 字 翔宇 ， 曾 用 名 飞飞 、 伍豪 、 少山 、 冠生 等 ， 原籍 浙江 绍兴 ， 1898年 3月5日 生 于 江苏 淮安 。 1921年 加入 中国 共产党 ， 是 伟大 的 马克思 主义 者 ， 伟大 的 无产阶级 革命 家 、 政治家 、 军事家 、 外交家 ， 党 和 国家 主要 领导 人 之一 ， 中国 人民 解放军 主要 创建 人 之一 ， 中华 人民 共和国 的 开国元勋 ， 是 以 毛泽东 同志 为 核心 的 党 的 第一 代 中央 领导 集体 的 重要 成员 。 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

分字###

In [7]:
from transformers import BertTokenizer, BertModel, BertForMaskedLM
import smoothnlp as snlp
import re
import torch

In [57]:
sentences = []

ubuntu_path = "/home/zoe/文档/6.10/train.txt"
win_path = "D:/数据集大荟萃/DIY/train.txt"

with open(ubuntu_path,'r',encoding="utf8") as f:
  sentences = f.readlines()

import re
chars_list = []
chars_str = []
masked_idxs_list = []
for sent in sentences:
    #句子预处理
    sent = sent.replace('\ufeff','')
    sent = sent.replace(' ','')
    sent = sent[1:]
    sent = sent.strip()
    
    chars = ' '.join(sent)
    chars_str.append(chars)
    chars_l = chars.split(" ")
    chars_list.append(chars_l)
    
    masked_idxs = []
    for i,ch in enumerate(chars_l):
        if ch=='#':
            masked_idxs.append(i+1)
    masked_idxs_list.append(masked_idxs)
    
# print("chars_list:",chars_list)
# print("chars_str:",chars_str)
# print("masked_idxs_list:",masked_idxs_list)

chars_list: [['#', '#', '#', '（', '1', '8', '9', '8', '年', '3', '月', '5', '日', '-', '1', '9', '7', '6', '年', '1', '月', '8', '日', '）', '，', '字', '翔', '宇', '，', '曾', '用', '名', '飞', '飞', '、', '伍', '豪', '、', '少', '山', '、', '冠', '生', '等', '，', '原', '籍', '浙', '江', '绍', '兴', '，', '1', '8', '9', '8', '年', '3', '月', '5', '日', '生', '于', '江', '苏', '淮', '安', '。', '1', '9', '2', '1', '年', '加', '入', '中', '国', '共', '产', '党', '，', '是', '伟', '大', '的', '马', '克', '思', '主', '义', '者', '，', '伟', '大', '的', '无', '产', '阶', '级', '革', '命', '家', '、', '政', '治', '家', '、', '军', '事', '家', '、', '外', '交', '家', '，', '党', '和', '国', '家', '主', '要', '领', '导', '人', '之', '一', '，', '中', '国', '人', '民', '解', '放', '军', '主', '要', '创', '建', '人', '之', '一', '，', '中', '华', '人', '民', '共', '和', '国', '的', '开', '国', '元', '勋', '，', '是', '以', '毛', '泽', '东', '同', '志', '为', '核', '心', '的', '党', '的', '第', '一', '代', '中', '央', '领', '导', '集', '体', '的', '重', '要', '成', '员', '。'], ['周', '恩', '来', '（', '1', '8', '9', '8', '年', '3', '月', '5', '日', '-',

In [58]:
max_sent_len = 512
chars_list_512,chars_str_512 = [], []

for chars_l,chars in zip(chars_list,chars_str):
#     print(len(chars_l),len(chars.split(' ')))
    assert len(chars_l)==len(chars.split(' '))
    if len(chars_l)<max_sent_len:
        chars_l_512 = ['[CLS]'] + chars_l + ['[SEP]'] + (max_sent_len-3-len(chars_l))*['[PAD]'] + ['[SEP]']
        #将#变为[MASK]
        for i,ch in enumerate(chars_l_512):
            if ch=='#':
                chars_l_512[i]='[MASK]'
        
        chars_512 = ' '.join(chars_l_512)
        chars_list_512.append(chars_l_512)
        chars_str_512.append(chars_512)
    
    
        
# for chars_l,chars in zip(chars_list,chars_str):
#     print(len(chars_l),len(chars.split(' ')))
# for chars_l_512,chars_512 in zip(chars_list_512,chars_str_512):
#     print(len(chars_l_512),len(chars_512.split(' ')))

In [52]:
for chars_l_512,chars_512 in zip(chars_list_512,chars_str_512):
    print(chars_l_512,chars_512)

['[CLS]', '[MASK]', '[MASK]', '[MASK]', '（', '1', '8', '9', '8', '年', '3', '月', '5', '日', '-', '1', '9', '7', '6', '年', '1', '月', '8', '日', '）', '，', '字', '翔', '宇', '，', '曾', '用', '名', '飞', '飞', '、', '伍', '豪', '、', '少', '山', '、', '冠', '生', '等', '，', '原', '籍', '浙', '江', '绍', '兴', '，', '1', '8', '9', '8', '年', '3', '月', '5', '日', '生', '于', '江', '苏', '淮', '安', '。', '1', '9', '2', '1', '年', '加', '入', '中', '国', '共', '产', '党', '，', '是', '伟', '大', '的', '马', '克', '思', '主', '义', '者', '，', '伟', '大', '的', '无', '产', '阶', '级', '革', '命', '家', '、', '政', '治', '家', '、', '军', '事', '家', '、', '外', '交', '家', '，', '党', '和', '国', '家', '主', '要', '领', '导', '人', '之', '一', '，', '中', '国', '人', '民', '解', '放', '军', '主', '要', '创', '建', '人', '之', '一', '，', '中', '华', '人', '民', '共', '和', '国', '的', '开', '国', '元', '勋', '，', '是', '以', '毛', '泽', '东', '同', '志', '为', '核', '心', '的', '党', '的', '第', '一', '代', '中', '央', '领', '导', '集', '体', '的', '重', '要', '成', '员', '。', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]

In [59]:
#加载词典
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

all_tokens_tensor = []
all_segments_tensors = []
for word_str in chars_list_512:
    # 将 token 转为 vocabulary 索引
    indexed_tokens = tokenizer.convert_tokens_to_ids(word_str)

    # 定义句子 A、B 索引
    segments_ids = [0]*len(word_str)
    
    # 将 inputs 转为 PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    
    all_tokens_tensor.append(tokens_tensor)
    all_segments_tensors.append(segments_tensors)

In [64]:
# 加载模型 pre-trained model (weights)
model = BertForMaskedLM.from_pretrained('bert-base-chinese')
model.eval()
 
for tokens_tensor,segments_tensors,masked_idxs in zip(all_tokens_tensor,all_segments_tensors,masked_idxs_list):
    # cuda
    tokens_tensor = tokens_tensor.to('cuda')
    segments_tensors = segments_tensors.to('cuda')
    model.to('cuda')

    # Predict all tokens
    with torch.no_grad():
        predictions = model(tokens_tensor, segments_tensors)
        
    print(masked_idxs)
    # confirm we were able to predict 'henson'
    for idx in masked_idxs:
        predicted_index = torch.argmax(predictions.logits[0, idx]).item()
        predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
        print(predicted_token)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[1, 2, 3]
刘
志
东
[158, 159, 160]
周
近
来
[48, 49, 50, 51]
江
苏
淮
州
[64, 65, 66, 67]
浙
江
绍
兴
[78, 79, 80]
共
产
党
[1, 2, 3]
王
晓
华
[36, 37, 38, 39]
1949
年
2
月
[70, 71, 72]
共
产
党
[41, 42, 43, 44]
1
9
2
4


bert做完形填空
正确答案:周恩来 毛泽东 浙江绍兴 江苏淮安 共产党 邓颖超 广西南宁 共产党 1924
bert答案:刘志东 周近来 江苏淮洲 浙江绍兴 共产党 王晓华 1949年2月 共产党 1924